In [0]:
#create spark session
from pyspark.sql import SparkSession

spark=SparkSession.builder.appName('nlp').getOrCreate()

In [0]:
df=spark.createDataFrame([(1,'I really liked this movie'),
                         (2,'I would recommend this movie to my friends'),
                         (3,'movie was alright but acting was horrible'),
                         (4,'I am never watching that movie ever again')],
                        ['user_id','review'])

In [0]:
df.show(5,False)

+-------+------------------------------------------+
|user_id|review                                    |
+-------+------------------------------------------+
|1      |I really liked this movie                 |
|2      |I would recommend this movie to my friends|
|3      |movie was alright but acting was horrible |
|4      |I am never watching that movie ever again |
+-------+------------------------------------------+



In [0]:
# Tokenization

In [0]:
from pyspark.ml.feature import Tokenizer

In [0]:
tokenization=Tokenizer(inputCol='review',outputCol='tokens')

In [0]:
tokenized_df=tokenization.transform(df)

In [0]:
tokenized_df.show(4,False)

+-------+------------------------------------------+---------------------------------------------------+
|user_id|review                                    |tokens                                             |
+-------+------------------------------------------+---------------------------------------------------+
|1      |I really liked this movie                 |[i, really, liked, this, movie]                    |
|2      |I would recommend this movie to my friends|[i, would, recommend, this, movie, to, my, friends]|
|3      |movie was alright but acting was horrible |[movie, was, alright, but, acting, was, horrible]  |
|4      |I am never watching that movie ever again |[i, am, never, watching, that, movie, ever, again] |
+-------+------------------------------------------+---------------------------------------------------+



In [0]:
# stopwords removal 

In [0]:
from pyspark.ml.feature import StopWordsRemover

In [0]:
stopword_removal=StopWordsRemover(inputCol='tokens',outputCol='refined_tokens')

In [0]:
refined_df=stopword_removal.transform(tokenized_df)

In [0]:
refined_df.select(['user_id','tokens','refined_tokens']).show(10,False)

+-------+---------------------------------------------------+----------------------------------+
|user_id|tokens                                             |refined_tokens                    |
+-------+---------------------------------------------------+----------------------------------+
|1      |[i, really, liked, this, movie]                    |[really, liked, movie]            |
|2      |[i, would, recommend, this, movie, to, my, friends]|[recommend, movie, friends]       |
|3      |[movie, was, alright, but, acting, was, horrible]  |[movie, alright, acting, horrible]|
|4      |[i, am, never, watching, that, movie, ever, again] |[never, watching, movie, ever]    |
+-------+---------------------------------------------------+----------------------------------+



In [0]:
# Count Vectorizer

In [0]:
from pyspark.ml.feature import CountVectorizer

In [0]:
count_vec=CountVectorizer(inputCol='refined_tokens',outputCol='features')

In [0]:
cv_df=count_vec.fit(refined_df).transform(refined_df)

In [0]:
cv_df.select(['user_id','refined_tokens','features']).show(4,False)

+-------+----------------------------------+---------------------------------+
|user_id|refined_tokens                    |features                         |
+-------+----------------------------------+---------------------------------+
|1      |[really, liked, movie]            |(11,[0,3,5],[1.0,1.0,1.0])       |
|2      |[recommend, movie, friends]       |(11,[0,7,8],[1.0,1.0,1.0])       |
|3      |[movie, alright, acting, horrible]|(11,[0,2,4,9],[1.0,1.0,1.0,1.0]) |
|4      |[never, watching, movie, ever]    |(11,[0,1,6,10],[1.0,1.0,1.0,1.0])|
+-------+----------------------------------+---------------------------------+



In [0]:
count_vec.fit(refined_df).vocabulary

Out[96]: ['movie',
 'ever',
 'liked',
 'really',
 'friends',
 'recommend',
 'watching',
 'horrible',
 'never',
 'acting',
 'alright']

In [0]:
#Tf-idf

In [0]:
from pyspark.ml.feature import HashingTF,IDF

In [0]:
hashing_vec=HashingTF(inputCol='refined_tokens',outputCol='tf_features')

In [0]:
hashing_df=hashing_vec.transform(refined_df)

In [0]:
hashing_df.select(['user_id','refined_tokens','tf_features']).show(4,False)

+-------+----------------------------------+-------------------------------------------------------+
|user_id|refined_tokens                    |tf_features                                            |
+-------+----------------------------------+-------------------------------------------------------+
|1      |[really, liked, movie]            |(262144,[99172,210223,229264],[1.0,1.0,1.0])           |
|2      |[recommend, movie, friends]       |(262144,[68228,130047,210223],[1.0,1.0,1.0])           |
|3      |[movie, alright, acting, horrible]|(262144,[95685,171118,210223,236263],[1.0,1.0,1.0,1.0])|
|4      |[never, watching, movie, ever]    |(262144,[63139,113673,203802,210223],[1.0,1.0,1.0,1.0])|
+-------+----------------------------------+-------------------------------------------------------+



In [0]:
tf_idf_vec=IDF(inputCol='tf_features',outputCol='tf_idf_features')

In [0]:
tf_idf_df=tf_idf_vec.fit(hashing_df).transform(hashing_df)

In [0]:
tf_idf_df.select(['user_id','tf_idf_features']).show(4,False)

+-------+----------------------------------------------------------------------------------------------------+
|user_id|tf_idf_features                                                                                     |
+-------+----------------------------------------------------------------------------------------------------+
|1      |(262144,[99172,210223,229264],[0.9162907318741551,0.0,0.9162907318741551])                          |
|2      |(262144,[68228,130047,210223],[0.9162907318741551,0.9162907318741551,0.0])                          |
|3      |(262144,[95685,171118,210223,236263],[0.9162907318741551,0.9162907318741551,0.0,0.9162907318741551])|
|4      |(262144,[63139,113673,203802,210223],[0.9162907318741551,0.9162907318741551,0.9162907318741551,0.0])|
+-------+----------------------------------------------------------------------------------------------------+



In [0]:
# Classification 

In [0]:
text_df=spark.read.csv('dbfs:/FileStore/tables/Movie_reviews.csv',inferSchema=True,header=True,sep=',')

In [0]:
text_df.printSchema()

root
 |-- Review: string (nullable = true)
 |-- Sentiment: string (nullable = true)



In [0]:
text_df.count()

Out[108]: 7087

In [0]:
from pyspark.sql.functions import rand 

In [0]:
text_df.orderBy(rand()).show(10,False)

+------------------------------------------------------------------------+---------+
|Review                                                                  |Sentiment|
+------------------------------------------------------------------------+---------+
|I love Harry Potter.                                                    |1        |
|I really hate the Da Vinci Code...                                      |0        |
|Da Vinci Code sucks.                                                    |0        |
|Da Vinci Code sucks.                                                    |0        |
|friday hung out with kelsie and we went and saw The Da Vinci Code SUCKED|0        |
|Love luv lubb the Da Vinci Code!                                        |1        |
|I love Harry Potter.                                                    |1        |
|I am going to start reading the Harry Potter series again because that i|1        |
|I, too, like Harry Potter..                                     

In [0]:
text_df=text_df.filter(((text_df.Sentiment =='1') | (text_df.Sentiment =='0')))

In [0]:
text_df.count()

Out[112]: 6990

In [0]:
text_df.groupBy('Sentiment').count().show()

+---------+-----+
|Sentiment|count|
+---------+-----+
|        0| 3081|
|        1| 3909|
+---------+-----+



In [0]:
text_df.printSchema()

root
 |-- Review: string (nullable = true)
 |-- Sentiment: string (nullable = true)



In [0]:
text_df = text_df.withColumn("Label", text_df.Sentiment.cast('float')).drop('Sentiment')

In [0]:
text_df.orderBy(rand()).show(10,False)

+---------------------------------------------------------------------+-----+
|Review                                                               |Label|
+---------------------------------------------------------------------+-----+
|Harry Potter dragged Draco Malfoy ’ s trousers down past his hips and|0.0  |
|man i loved brokeback mountain!                                      |1.0  |
|Mission Impossible 3 was excellent.                                  |1.0  |
|Ok brokeback mountain is such a horrible movie.                      |0.0  |
|I love Brokeback Mountain.                                           |1.0  |
|i love mission impossible!!!!!!!!!!!..                               |1.0  |
|we're gonna like watch Mission Impossible or Hoot.(                  |1.0  |
|Da Vinci Code sucks be...                                            |0.0  |
|dudeee i LOVED brokeback mountain!!!!                                |1.0  |
|Brokeback Mountain was an awesome movie.                       

In [0]:
text_df.groupBy('label').count().show()

+-----+-----+
|label|count|
+-----+-----+
|  1.0| 3909|
|  0.0| 3081|
+-----+-----+



In [0]:
# Add length to the dataframe
from pyspark.sql.functions import length

In [0]:
text_df=text_df.withColumn('length',length(text_df['Review']))

In [0]:
text_df.orderBy(rand()).show(10,False)

+------------------------------------------------------------------------+-----+------+
|Review                                                                  |Label|length|
+------------------------------------------------------------------------+-----+------+
|My dad's being stupid about brokeback mountain...                       |0.0  |49    |
|Is it just me, or does Harry Potter suck?...                            |0.0  |44    |
|Brokeback Mountain was an AWESOME movie.                                |1.0  |40    |
|I, too, like Harry Potter..                                             |1.0  |27    |
|The Da Vinci Code was absolutely AWESOME!                               |1.0  |41    |
|by the way, the Da Vinci Code sucked, just letting you know...          |0.0  |62    |
|I am the only person in the world who thought Brokeback Mountain sucked.|0.0  |72    |
|I hate Harry Potter, it's retarted, gay and stupid and there's only one |0.0  |71    |
|Is it just me, or does Harry Po

In [0]:
text_df.groupBy('Label').agg({'Length':'mean'}).show()

+-----+-----------------+
|Label|      avg(Length)|
+-----+-----------------+
|  1.0|47.61882834484523|
|  0.0|50.95845504706264|
+-----+-----------------+



In [0]:
# Data Cleaning

In [0]:
tokenization=Tokenizer(inputCol='Review',outputCol='tokens')

In [0]:
tokenized_df=tokenization.transform(text_df)

In [0]:
tokenized_df.show()

+--------------------+-----+------+--------------------+
|              Review|Label|length|              tokens|
+--------------------+-----+------+--------------------+
|The Da Vinci Code...|  1.0|    39|[the, da, vinci, ...|
|this was the firs...|  1.0|    72|[this, was, the, ...|
|i liked the Da Vi...|  1.0|    32|[i, liked, the, d...|
|i liked the Da Vi...|  1.0|    32|[i, liked, the, d...|
|I liked the Da Vi...|  1.0|    72|[i, liked, the, d...|
|that's not even a...|  1.0|    72|[that's, not, eve...|
|I loved the Da Vi...|  1.0|    72|[i, loved, the, d...|
|i thought da vinc...|  1.0|    57|[i, thought, da, ...|
|The Da Vinci Code...|  1.0|    45|[the, da, vinci, ...|
|I thought the Da ...|  1.0|    51|[i, thought, the,...|
|The Da Vinci Code...|  1.0|    68|[the, da, vinci, ...|
|The Da Vinci Code...|  1.0|    62|[the, da, vinci, ...|
|then I turn on th...|  1.0|    66|[then, i, turn, o...|
|The Da Vinci Code...|  1.0|    34|[the, da, vinci, ...|
|i love da vinci c...|  1.0|   

In [0]:
stopword_removal=StopWordsRemover(inputCol='tokens',outputCol='refined_tokens')

In [0]:
refined_text_df=stopword_removal.transform(tokenized_df)

In [0]:
refined_text_df.show()

+--------------------+-----+------+--------------------+--------------------+
|              Review|Label|length|              tokens|      refined_tokens|
+--------------------+-----+------+--------------------+--------------------+
|The Da Vinci Code...|  1.0|    39|[the, da, vinci, ...|[da, vinci, code,...|
|this was the firs...|  1.0|    72|[this, was, the, ...|[first, clive, cu...|
|i liked the Da Vi...|  1.0|    32|[i, liked, the, d...|[liked, da, vinci...|
|i liked the Da Vi...|  1.0|    32|[i, liked, the, d...|[liked, da, vinci...|
|I liked the Da Vi...|  1.0|    72|[i, liked, the, d...|[liked, da, vinci...|
|that's not even a...|  1.0|    72|[that's, not, eve...|[even, exaggerati...|
|I loved the Da Vi...|  1.0|    72|[i, loved, the, d...|[loved, da, vinci...|
|i thought da vinc...|  1.0|    57|[i, thought, da, ...|[thought, da, vin...|
|The Da Vinci Code...|  1.0|    45|[the, da, vinci, ...|[da, vinci, code,...|
|I thought the Da ...|  1.0|    51|[i, thought, the,...|[thought

In [0]:
from pyspark.sql.functions import udf
from pyspark.sql.types import IntegerType
from pyspark.sql.functions import *

In [0]:
len_udf = udf(lambda s: len(s), IntegerType())

refined_text_df = refined_text_df.withColumn("token_count", len_udf(col('refined_tokens')))


In [0]:
refined_text_df.orderBy(rand()).show(10)

+--------------------+-----+------+--------------------+--------------------+-----------+
|              Review|Label|length|              tokens|      refined_tokens|token_count|
+--------------------+-----+------+--------------------+--------------------+-----------+
|I love The Da Vin...|  1.0|    27|[i, love, the, da...|[love, da, vinci,...|          4|
|BROKEBACK MOUNTAI...|  0.0|    27|[brokeback, mount...|[brokeback, mount...|          3|
|Mission Impossibl...|  1.0|    60|[mission, impossi...|[mission, impossi...|          8|
|I love Brokeback ...|  1.0|    26|[i, love, brokeba...|[love, brokeback,...|          3|
|Is it just me, or...|  0.0|    44|[is, it, just, me...|[me,, harry, pott...|          4|
|So Brokeback Moun...|  0.0|    44|[so, brokeback, m...|[brokeback, mount...|          4|
|dudeee i LOVED br...|  1.0|    37|[dudeee, i, loved...|[dudeee, loved, b...|          4|
|The Da Vinci Code...|  1.0|    30|[the, da, vinci, ...|[da, vinci, code,...|          4|
|Then snuc

In [0]:
count_vec=CountVectorizer(inputCol='refined_tokens',outputCol='features')

In [0]:
cv_text_df=count_vec.fit(refined_text_df).transform(refined_text_df)

In [0]:
cv_text_df.select(['refined_tokens','token_count','features','Label']).show(10)

+--------------------+-----------+--------------------+-----+
|      refined_tokens|token_count|            features|Label|
+--------------------+-----------+--------------------+-----+
|[da, vinci, code,...|          5|(2302,[0,1,4,43,2...|  1.0|
|[first, clive, cu...|          9|(2302,[11,51,229,...|  1.0|
|[liked, da, vinci...|          5|(2302,[0,1,4,52,3...|  1.0|
|[liked, da, vinci...|          5|(2302,[0,1,4,52,3...|  1.0|
|[liked, da, vinci...|          8|(2302,[0,1,4,52,7...|  1.0|
|[even, exaggerati...|          6|(2302,[46,229,272...|  1.0|
|[loved, da, vinci...|          8|(2302,[0,1,22,30,...|  1.0|
|[thought, da, vin...|          7|(2302,[0,1,4,228,...|  1.0|
|[da, vinci, code,...|          6|(2302,[0,1,4,33,2...|  1.0|
|[thought, da, vin...|          7|(2302,[0,1,4,223,...|  1.0|
+--------------------+-----------+--------------------+-----+
only showing top 10 rows



In [0]:
#select data for building model
model_text_df=cv_text_df.select(['features','token_count','Label'])

In [0]:
from pyspark.ml.feature import VectorAssembler

In [0]:
df_assembler = VectorAssembler(inputCols=['features','token_count'],outputCol='features_vec')
model_text_df = df_assembler.transform(model_text_df)

In [0]:
model_text_df.printSchema()

root
 |-- features: vector (nullable = true)
 |-- token_count: integer (nullable = true)
 |-- Label: float (nullable = true)
 |-- features_vec: vector (nullable = true)



In [0]:
from pyspark.ml.classification import LogisticRegression

In [0]:
#split the data 
training_df,test_df=model_text_df.randomSplit([0.75,0.25])

In [0]:
training_df.groupBy('Label').count().show()

+-----+-----+
|Label|count|
+-----+-----+
|  1.0| 2954|
|  0.0| 2315|
+-----+-----+



In [0]:
test_df.groupBy('Label').count().show()

+-----+-----+
|Label|count|
+-----+-----+
|  1.0|  955|
|  0.0|  766|
+-----+-----+



In [0]:
log_reg=LogisticRegression(featuresCol='features_vec',labelCol='Label').fit(training_df)

In [0]:
results=log_reg.evaluate(test_df).predictions

In [0]:
results.show()

+--------------------+-----------+-----+--------------------+--------------------+--------------------+----------+
|            features|token_count|Label|        features_vec|       rawPrediction|         probability|prediction|
+--------------------+-----------+-----+--------------------+--------------------+--------------------+----------+
|(2302,[0,1,4,5,91...|          9|  1.0|(2303,[0,1,4,5,91...|[-7.9353484422111...|[3.57739229770268...|       1.0|
|(2302,[0,1,4,10,2...|          7|  0.0|(2303,[0,1,4,10,2...|[25.0754395925269...|[0.99999999998712...|       0.0|
|(2302,[0,1,4,11,1...|          6|  0.0|(2303,[0,1,4,11,1...|[20.3890298053025...|[0.99999999860312...|       0.0|
|(2302,[0,1,4,12,1...|          5|  1.0|(2303,[0,1,4,12,1...|[-18.281416025188...|[1.14942832176987...|       1.0|
|(2302,[0,1,4,12,1...|          5|  1.0|(2303,[0,1,4,12,1...|[-18.281416025188...|[1.14942832176987...|       1.0|
|(2302,[0,1,4,12,3...|          5|  1.0|(2303,[0,1,4,12,3...|[-16.800362564402..

In [0]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator


In [0]:
#confusion matrix
true_postives = results[(results.Label == 1) & (results.prediction == 1)].count()
true_negatives = results[(results.Label == 0) & (results.prediction == 0)].count()
false_positives = results[(results.Label == 0) & (results.prediction == 1)].count()
false_negatives = results[(results.Label == 1) & (results.prediction == 0)].count()

In [0]:
recall = float(true_postives)/(true_postives + false_negatives)
print(recall)

0.9926701570680628


In [0]:
precision = float(true_postives) / (true_postives + false_positives)
print(precision)

0.977319587628866


In [0]:
accuracy=float((true_postives+true_negatives) /(results.count()))
print(accuracy)

0.9831493317838466
